In [ ]:
#Useful imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2
import cv2
from refinement import get_vegetation_masks
from plots import plot_picture
from refinement import python2matlab
from refinement import ycc2rgb, y_python2matlab
from pixel_fusion import pixel_fusion
from multiresolution_decomposition import multiresolution_synthesis

In [ ]:
def dehaze_with_mask(rgb_detail, nir_detail, rgb_base_image, matlab_img_ycrcb, v_mask=None, filename=''):
   
    fused = pixel_fusion((rgb_detail, nir_detail),v_mask, False)
    image_dehazed = multiresolution_synthesis((rgb_base_image, fused))
    Y_fused = np.round(image_dehazed * 255)
    Y_fused = y_python2matlab(Y_fused)
    img_rgb_dehazed = ycc2rgb(Y_fused,matlab_img_ycrcb)
    plt.imshow(img_rgb_dehazed)
    plt.show()
    if filename != '':
        img_bgr_dehazed = cv2.cvtColor(img_rgb_dehazed, cv2.COLOR_RGB2BGR)
        cv2.imwrite(filename, img_bgr_dehazed)
    return img_rgb_dehazed

registration = False
filenames = [['branchview','.tif'],
             ['knoll','.tif']]            


for arg in filenames:
    filename, filetype = arg
    rgb_path = 'input/' + filename + filetype
    nir_path = 'input/' + filename + '_nir' + filetype

    ##   read images
    img_gray = cv2.imread(rgb_path, 0)
    img_bgr = cv2.imread(rgb_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    img_lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    img_ycrcb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2YCrCb)
    img_lum = img_lab[..., 0]
    img_nir_old = cv2.imread(nir_path)[..., 0]
    
    if (registration):
        ##  image registration
        from registration import *
        img_nir = registration(img_gray, img_nir_old, plot=True)
        cv2.imwrite('output/' + filename + '_registered.png', img_nir)
    else:
        img_nir = img_nir_old
        
    rgb_base = []
    nir_base = []
    rgb_detail = []
    nir_detail = []
    n_base = 7
    n_detail = 6 
    for counter in range(n_base):
        nir_base.append(np.load('output/{}_nir_base{}.npy'.format(filename,counter)))
        rgb_base.append(np.load('output/{}_vis_base{}.npy'.format(filename,counter)))
    for counter in range(n_detail):
        nir_detail.append(np.load('output/{}_nir_detail{}.npy'.format(filename,counter)))
        rgb_detail.append(np.load('output/{}_vis_detail{}.npy'.format(filename,counter)))

        red = (img_rgb[:,:,0]/255.0).astype(np.float32)
    nir = (img_nir/255.0).astype(np.float32)
    v_mask, v_prob = get_vegetation_masks(red, nir)
    np.save('dehazed/'+filename+'_vmask',v_mask)
    np.save('dehazed/'+filename+'_vprob',v_prob)
    matlab_img_ycrcb = python2matlab(img_ycrcb)

    img_rgb_dehazed_binary = dehaze_with_mask(rgb_detail, nir_detail, rgb_base[-1], matlab_img_ycrcb, 
                                              v_mask=v_mask, filename='dehazed/'+filename+'_dehazed_binary.png')
    img_rgb_dehazed_prob = dehaze_with_mask(rgb_detail, nir_detail, rgb_base[-1], matlab_img_ycrcb, 
                                            v_mask=v_prob, filename='dehazed/'+filename+'_dehazed_prob.png')
    img_rgb_dehazed = dehaze_with_mask(rgb_detail, nir_detail, rgb_base[-1], matlab_img_ycrcb, 
                                       v_mask=None, filename='dehazed/'+filename+'_dehazed.png')


## plot tiles

In [ ]:
def tile_pictures(picture_array, location):
    xmin, xmax, ymin, ymax = location
    first = True
    for pic in picture_array:
        if first:
            new_picture = pic[ymin:ymax,xmin:xmax]
            first = False
        else:
            new_picture = np.concatenate((new_picture, pic[ymin:ymax,xmin:xmax]), axis=1)
    return new_picture

for arg in filenames:
    filename, filetype = arg
    rgb_path = 'input/' + filename + filetype
    print('file',filename)
    ##   read images
    print('reading raw images')
    img_bgr = cv2.imread(rgb_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    
    height, width, __ = img_rgb.shape
    tile_size = 1000
    location = (width-tile_size,width,height-tile_size,height)

    v_mask_path='dehazed/'+filename+'_vmask.npy'
    v_prob_path='dehazed/'+filename+'_vprob.npy'
    dehazed_binary_path='dehazed/'+filename+'_dehazed_binary.png'
    dehazed_prob_path='dehazed/'+filename+'_dehazed_prob.png'
    dehazed_path='dehazed/'+filename+'_dehazed.png'
    
    print('reading masks')
    v_mask = np.load(v_mask_path)
    v_prob = np.load(v_prob_path)
    
    print('reading dehazed')
    img_rgb_dehazed_binary = cv2.imread(dehazed_binary_path)
    img_rgb_dehazed_prob = cv2.imread(dehazed_prob_path)
    img_rgb_dehazed = cv2.imread(dehazed_path)
    
    img_rgb_dehazed_binary = cv2.cvtColor(img_rgb_dehazed_binary, cv2.COLOR_BGR2RGB)
    img_rgb_dehazed_prob = cv2.cvtColor(img_rgb_dehazed_prob, cv2.COLOR_BGR2RGB)
    img_rgb_dehazed = cv2.cvtColor(img_rgb_dehazed, cv2.COLOR_BGR2RGB)

    plot_picture(v_mask,'vmask',colorbar='in', filename='dehazed/'+filename+'_vmask.pdf')
    plot_picture(v_prob,'vprob',colorbar='in', filename='dehazed/'+filename+'_vprob.pdf')
    
    masks = tile_pictures([v_mask, v_prob], location)
    diff = img_rgb - img_rgb_dehazed_binary
    results_binary = tile_pictures([img_rgb, img_rgb_dehazed, img_rgb_dehazed_binary, diff], location)
    diff = img_rgb - img_rgb_dehazed_prob
    results_prob = tile_pictures([img_rgb, img_rgb_dehazed, img_rgb_dehazed_prob, diff], location)
    plot_picture(masks,colorbar='out',filename='output/'+filename+'_masks.pdf')
    plot_picture(results_binary,colorbar=False,filename='output/'+filename+'_results_binary.pdf')
    plot_picture(results_prob,colorbar=False,filename='output/'+filename+'_results_prob.pdf')